<a href="https://colab.research.google.com/github/ArshHp/LLM/blob/main/multiLingual_ctxt_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["GROQ_API_KEY"]="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"

In [2]:
!pip install -q langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.2 MB/s eta 0:00:00


In [4]:
!pip install -q langchain langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [6]:
!pip install -q langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [21]:
from langchain.prompts import PromptTemplate
from langdetect import detect

# Define the reusable prompt template
prompt = PromptTemplate.from_template("""
### System Instruction:
You are a helpful assistant that always replies in the **same language** as the user's question,
even when the reference information is in other languages.
Always prioritize the user query's language for your response.

### Language Hint:
User query language: {language}

### Reference Context:
[REF]
{reference}
[/REF]

### User Question:
{question}

### Your Response (in the user's language):
""")




In [15]:
from langdetect import detect
detect_language = RunnableLambda(lambda x: {**x, "language": detect(x["question"])})

In [16]:
#Option 1
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")

#Option 2 : Load the LLAMA 8B model in GPU, ON CPU Below code will not fly well
#from langchain_community.llms import HuggingFacePipeline  # Replace as needed
#from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
#hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
#llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [17]:
from langchain_core.runnables import RunnableLambda, RunnableMap
# Create the LCEL pipeline
chain = (
    detect_language
    | RunnableMap({
        "language": lambda x: x["language"],
        "reference": lambda x: x["reference"],
        "question": lambda x: x["question"],
    })
    |
    prompt
    |
    llm
)


In [18]:
input_data = {
    "question": "Há alguma mudança planeada para os serviços SAP?",
    "reference": """
Desde quando tem este problema? Apenas se passa consigo? Tentou reiniciar o seu computador?
Something changed? Did you update your OS? Demander une image de SE.
"""
}


In [40]:
from IPython.display import Markdown, display
response = chain.invoke(input_data)
display(Markdown(response.content))


Não há informações disponíveis sobre mudanças planeadas nos serviços SAP na referência fornecida. No entanto, posso sugerir algumas perguntas para ajudar a encontrar mais informações:

- Você está usando alguma versão específica do SAP?
- Há algum problema específico com os serviços SAP que você gostaria de resolver?
- Você tem acesso a informações de suporte ou documentação sobre os serviços SAP?

Se você tiver mais informações, posso tentar ajudar a encontrar as respostas que você está procurando.

In [41]:
# 🔹 Inference function for Gradio
def multilingual_agent(question, reference):
    input_data = {
        "question": question,
        "reference": reference
    }
    response = chain.invoke(input_data)
    return response.content

In [22]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00


In [23]:
import gradio as gr

In [43]:
# 🔹 Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🌍 Multilingual Assistant (Using LLaMA 8B)")
    gr.Markdown("Ask your question in any language. The assistant will respond in the same language, using mixed-language context.")

    with gr.Row():
        question = gr.Textbox(label="Your Question", placeholder="Há alguma mudança planeada para os serviços SAP?")
    reference = gr.Textbox(lines=6, label="Reference Context (optional multilingual)", placeholder="Paste any multilingual context here...")

    output = gr.Textbox(lines=6, label="LLM Response")

    run_button = gr.Button("Generate Response")

    run_button.click(fn=multilingual_agent, inputs=[question, reference], outputs=output)

# 🔹 Launch the app
if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e2e0022ed123ec2b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
